In [1]:
from functools import reduce
import json
import os.path
import sys

import matplotlib.pyplot as plt
from imageio import imread

import pandas as pd

sys.path.append(r'C:\Users\hornu\OneDrive\Documents\repos\flags\scripts\image-scripts')
from image_scaler_cleaner import ImageScalerCleaner

In [2]:
p_flag = r'C:\Users\hornu\OneDrive\Documents\repos\flags\data\clean-data\flag-details.csv'
p_codes = r'C:\Users\hornu\OneDrive\Documents\repos\flags\data\clean-data\clean-country-codes.csv'
p_religion = r'C:\Users\hornu\OneDrive\Documents\repos\flags\data\clean-data\clean-religions.csv'
p_independence = r'C:\Users\hornu\OneDrive\Documents\repos\flags\data\clean-data\clean-independence.csv'

df_flag = pd.read_csv(p_flag, sep = '\t')
df_codes = pd.read_csv(p_codes, sep = '\t', keep_default_na = False)
df_religion = pd.read_csv(p_religion, sep = '\t')
df_independence = pd.read_csv(p_independence, sep = '\t')

df_flag.index = df_flag.index.str.upper()

dfs = [df_flag, df_codes, df_religion, df_independence]

In [3]:
#  check for missing countries or inconsistently named ones
for idx1, df1 in enumerate(dfs):
    for idx2, df2 in enumerate(dfs[idx1+1:]):
        print(idx1, idx2 + idx1 + 1, set(df1.index).symmetric_difference(set(df2.index)))

0 1 set()
0 2 set()
0 3 set()
1 2 set()
1 3 set()
2 3 set()


In [4]:
df_merged = pd.concat(dfs, axis = 1, sort = False)

# create dict of countries
countries_dict = df_merged.to_dict(orient = 'index')

In [5]:
# read land and maritime borders

with open(r'C:\Users\hornu\OneDrive\Documents\repos\flags\data\clean-data\lbcc.json', 'r') as fproc:
    lb_dict = json.load(fproc)
    lb_dict = {k : list(v.keys()) for k, v in lb_dict.items()}
    
with open(r'C:\Users\hornu\OneDrive\Documents\repos\flags\data\clean-data\mbcc.json', 'r') as fproc:
    mb_dict = json.load(fproc)

In [6]:
# merge land maritime borders

ab_dict = dict(mb_dict.items())

for k, v in lb_dict.items():
    if k in ab_dict:
        ab_dict[k].extend([x for x in v if not x in ab_dict[k]])
    else:
        ab_dict.update({k : v})

In [7]:
# merge with neighbours
# remove unwanted fields

for k, v in countries_dict.items():
    v.update({'neighbours' : ab_dict[v['code']]})
    _ = v.pop('codeint')
    _ = v.pop('code3')
    _ = v.pop('power')

In [8]:
with open(r'C:\Users\hornu\OneDrive\Documents\repos\flags\data\clean-data\countries-master.json' , 'w') as fproc:
    json.dump(countries_dict, fproc, indent = 4)

In [5]:
with open(r'C:\Users\hornu\OneDrive\Documents\repos\flags\data\clean-data\clean-countries-master.json', 'r') as fproc:
    country_dict = json.load(fproc)

In [6]:
country_list = [x[1] for x in ((v.update({'name' : k}), v) for k, v in country_dict.items())]

In [7]:
import pymongo

In [4]:
mg_client = pymongo.MongoClient("mongodb://localhost:27017/")
mg_client.list_database_names()

['admin', 'config', 'flags', 'local']

In [8]:
# create database

db_flag = mg_client["flags"]
cl_countries_raw = db_flag["raw_countries"]
id_docs = cl_countries_raw.insert_many(country_list)


In [46]:
mg_client["flgas"]

TypeError: close_cursor() missing 1 required positional argument: 'cursor_id'